<a href="https://colab.research.google.com/github/MarielaNina/NeuralNetworksClass/blob/main/Projeto5_Mariela_Nina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projeto 5

• Implementar uma rede CNN

– Usar MNIST, MNIST-Fashion, ou CIFAR-10 (ou outro dataset)

• Avaliar diferentes topologias (>5 topologias) e parâmetros

• Calcular a matriz de confusão para os dois melhores modelos

• Avaliar a diferença entre os modelos (top-2)

• Treinar uma rede MLP (já implementada no trabalho anterior) e
comparar com a rede CNN gerada: acurácia versus número de
parâmetros